<a href="https://colab.research.google.com/github/matheusvazdata/dados-lab/blob/main/Introduction_to_APIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔄 Criando um Pipeline ETL com API

Um Engenheiro de Dados pode criar um pipeline ETL (Extract, Transform, Load) para armazenar dados de uma API em um banco de dados.

### 📌 Passo 1: Coletar Dados de uma API

In [1]:
import requests
import pandas as pd

# Fazendo a requisição na API
url = "https://economia.awesomeapi.com.br/json/last/USD-BRL,EUR-BRL,BTC-BRL"
response = requests.get(url)
dados_moeda = response.json()

# Verificando o status da requisição
if response.status_code == 200:
    print("Dados coletados com sucesso!")
else:
    print("Erro ao coletar dados da API.")

Dados coletados com sucesso!


In [2]:
# Convertendo para um DataFrame pandas
moedas = {
    "moeda": ["Dólar", "Euro", "Bitcoin"],
    "valor": [
        round(float(dados_moeda["USDBRL"]["bid"]), 2),
        round(float(dados_moeda["EURBRL"]["bid"]), 2),
        round(float(dados_moeda["BTCBRL"]["bid"]), 2)
    ]
}

df_moedas = pd.DataFrame(moedas)

# Exibindo os dados
df_moedas

,moeda,valor
0,Dólar,5.65
1,Euro,6.16
2,Bitcoin,488229.00


### 📌 Passo 2: Salvando os Dados no Banco de Dados

In [3]:
import sqlite3

# Criando conexão com SQLite
conn = sqlite3.connect("financeiro.db")
cursor = conn.cursor()

In [4]:
# Criando tabela
cursor.execute("""
CREATE TABLE IF NOT EXISTS cotacoes (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    moeda TEXT,
    valor REAL
)
""")

# Inserindo dados no banco
df_moedas.to_sql("cotacoes", conn, if_exists="append", index=False)

# Fechando conexão
conn.close()
print("Dados inseridos no banco de dados com sucesso!")

Dados inseridos no banco de dados com sucesso!


In [5]:
# Conexão (novamente) com o database, pois a conexão foi fechada anteriormente
conn = sqlite3.connect('financeiro.db')
cursor = conn.cursor()

# Escrita da query
cursor.execute("SELECT * FROM cotacoes")

# Exibe os resultados
for row in cursor.fetchall():
    print(row)

# Fechando conexão
conn.close()

(1, 'Dólar', 5.65)
(2, 'Euro', 6.16)
(3, 'Bitcoin', 488229.0)
